<a href="https://colab.research.google.com/github/yw228/CS452/blob/main/SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=a93874b44037ef6d5ecbc6754d557e2c80800f0f59972c575793e5acf00ca27b
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [7]:
# Mount my Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
spark = SparkSession.builder \
    .appName("SparkSQLExercise") \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')
sc.uiWebUrl

'http://d14768091729:4040'

In [9]:
myfile = "drive/MyDrive/Colab Notebooks/cs452/inputData/us-countries.csv"

data = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(myfile)
#df = spark.read.csv(myfile, header=True, inferSchema=True)



In [30]:
data.show()

+-------------------+-----------+----------+-----+-----+------+
|               date|     county|     state| fips|cases|deaths|
+-------------------+-----------+----------+-----+-----+------+
|2020-01-21 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-22 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-23 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-24 00:00:00|       Cook|  Illinois|17031|    1|     0|
|2020-01-24 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-25 00:00:00|     Orange|California| 6059|    1|     0|
|2020-01-25 00:00:00|       Cook|  Illinois|17031|    1|     0|
|2020-01-25 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-26 00:00:00|   Maricopa|   Arizona| 4013|    1|     0|
|2020-01-26 00:00:00|Los Angeles|California| 6037|    1|     0|
|2020-01-26 00:00:00|     Orange|California| 6059|    1|     0|
|2020-01-26 00:00:00|       Cook|  Illinois|17031|    1|     0|
|2020-01-26 00:00:00|  Snohomish|Washing

In [10]:
# Assuming the county column is named "county"
num_counties = data.select("county").distinct().count()

# Print the number of distinct counties
print("Number of counties:", num_counties)

Number of counties: 1932


In [13]:
# Assuming the county column is named "county"
num_states = data.select("state").distinct().count()

# Print the number of distinct counties
print("Number of States:", num_states)

Number of States: 56


2.Write code to find the county with the most deaths

In [15]:
latest_date = data.agg(max("date")).collect()[0][0]
latest_data = data.filter(data.date == latest_date)


latest_deaths = latest_data.groupBy("county") \
    .agg(max("deaths").alias("LatestDeaths"))


# latest_deaths.show(100)


max_deaths_county = latest_deaths.orderBy(desc("LatestDeaths")).first()

print("The county with the most deaths is {} with a total of {} deaths.".format(max_deaths_county.county, max_deaths_county.LatestDeaths))


The county with the most deaths is New York City with a total of 40267 deaths.


3.Write code to find the county with the most cases

In [55]:
latest_deaths = latest_data.groupBy("county") \
    .agg(max("cases").alias("LatestCases"))

#latest_deaths.show(100)

max_cases_county = latest_deaths.orderBy(desc("LatestCases")).first()

print("The county with the most casess is {} with a total of {} cases.".format(max_cases_county.county, max_cases_county.LatestCases))

The county with the most casess is Los Angeles with a total of 2908425 cases.


4.Write code to find the total number of deaths in Utah county

In [71]:
utah_data = data.filter((data.county == "Utah") & (data.state == "Utah"))

#utah_data.show()

latest_date = utah_data.agg(max("date")).collect()[0][0]
latest_data = utah_data.filter(data.date == latest_date)

latest_data.show()

total_deaths = latest_data.agg(sum("deaths")).collect()[0][0]
print("The total number of deaths in Utah county is {}".format(total_deaths))

+-------------------+------+-----+-----+------+------+
|               date|county|state| fips| cases|deaths|
+-------------------+------+-----+-----+------+------+
|2022-05-13 00:00:00|  Utah| Utah|49049|203286|   791|
+-------------------+------+-----+-----+------+------+

The total number of deaths in Utah county is 791


5. Write code to find the death rate for each state and sort the states by death rate

In [74]:
df.show(100)

+-------------------+-------------+-------------+-----+-----+------+
|               date|       county|        state| fips|cases|deaths|
+-------------------+-------------+-------------+-----+-----+------+
|2020-01-21 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-22 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-23 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-24 00:00:00|         Cook|     Illinois|17031|    1|     0|
|2020-01-24 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-25 00:00:00|       Orange|   California| 6059|    1|     0|
|2020-01-25 00:00:00|         Cook|     Illinois|17031|    1|     0|
|2020-01-25 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-26 00:00:00|     Maricopa|      Arizona| 4013|    1|     0|
|2020-01-26 00:00:00|  Los Angeles|   California| 6037|    1|     0|
|2020-01-26 00:00:00|       Orange|   California| 6059|    1|     0|
|2020-01-26 00:00:00|         Cook

In [19]:
from pyspark.sql.functions import round

#latest_data.show(100)


total_Death_df = latest_data.groupBy("state").agg(((sum("deaths")/sum("cases"))*100).alias("Death rate") )
total_Death_df = total_Death_df.withColumn("Death rate", (round(total_Death_df["Death rate"], 2))).orderBy(desc("Death rate"))
total_Death_df.show(100) 



+--------------------+----------+
|               state|Death rate|
+--------------------+----------+
|        Pennsylvania|      1.57|
|         Mississippi|      1.55|
|             Alabama|       1.5|
|             Georgia|      1.49|
|              Nevada|      1.49|
|             Arizona|      1.49|
|            Michigan|      1.46|
|          New Mexico|      1.45|
|          New Jersey|      1.45|
|            Missouri|      1.42|
|                Ohio|      1.42|
|         Connecticut|       1.4|
|           Louisiana|      1.39|
|            Oklahoma|      1.38|
|             Indiana|      1.38|
|            Maryland|      1.38|
|       West Virginia|      1.36|
|            Arkansas|      1.36|
|           Tennessee|      1.31|
|               Texas|       1.3|
|            New York|      1.29|
|             Florida|      1.24|
|                Iowa|      1.24|
|             Montana|      1.23|
|        South Dakota|      1.22|
|      South Carolina|      1.21|
|            I

6.Write code to something else interesting with this data – your choice

Find the State that has the most death

In [20]:
latest_date = data.agg(max("date")).collect()[0][0]
latest_data = data.filter(data.date == latest_date)


latest_deaths = latest_data.groupBy("state") \
    .agg(max("deaths").alias("LatestDeaths"))


# latest_deaths.show(100)


max_deaths_States = latest_deaths.orderBy(desc("LatestDeaths")).first()

print("The state with the most deaths is {} with a total of {} deaths.".format(max_deaths_States.state, max_deaths_States.LatestDeaths))

The state with the most deaths is New York with a total of 40267 deaths.
